### Context-based space filling curve (as per ChatGPT)

In [43]:
import numpy as np
import cv2
import networkx as nx
import matplotlib.pyplot as plt

def load_image(image_path, grayscale=True):
    """Load an image as a grid of pixel intensities."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR)
    return img

def construct_dual_graph(image):
    """Construct a dual graph with neighbors connected by pixel intensity differences."""
    rows, cols = image.shape
    G = nx.Graph()
    
    for r in range(rows):
        for c in range(cols):
            if c < cols - 1:  # Right neighbor
                G.add_edge((r, c), (r, c + 1), weight=abs(int(image[r, c]) - int(image[r, c + 1])))
            if r < rows - 1:  # Bottom neighbor
                G.add_edge((r, c), (r + 1, c), weight=abs(int(image[r, c]) - int(image[r + 1, c])))
    return G

def compute_minimum_spanning_tree(graph):
    """Compute the Minimum Spanning Tree (MST) of a graph using Prim's algorithm."""
    mst = nx.minimum_spanning_tree(graph, algorithm='prim')
    return mst

def visualize_graph(mst, image_shape):
    """Visualize the MST for debugging purposes."""
    pos = {node: (node[1], -node[0]) for node in mst.nodes()}
    nx.draw(mst, pos, with_labels=True, node_size=30, font_size=8, font_color='red', edge_color='blue')
    plt.title("MST Visualization")
    plt.show()

def manhattan_traversal(mst, start_node=(0, 0)):
    """Traverse MST using strict Manhattan movement with 90-degree turns."""
    visited = set()
    path = []
    
    # Start with the first node
    current_node = start_node
    path.append(current_node)
    visited.add(current_node)

    # Traverse MST following edges
    while len(visited) < len(mst.nodes):  # Ensure all nodes are visited
        # Get all unvisited neighbors of the current node
        possible_neighbors = [n for n in mst.neighbors(current_node) if n not in visited]
        
        if not possible_neighbors:
            break  # No more unvisited neighbors to explore
        
        # Pick the first unvisited neighbor
        best_neighbor = possible_neighbors[0]  
        
        # Add the neighbor to the path
        current_node = best_neighbor
        path.append(current_node)
        visited.add(current_node)
    
    return path

def plot_path(image, path):
    """Plot the image with a Manhattan traversal path."""
    plt.imshow(image, cmap='gray')
    x, y = zip(*path)
    plt.plot(y, x, color='red', linewidth=1)
    plt.title("Manhattan Space-Filling Curve")
    plt.axis("off")
    plt.show()



In [ ]:
# Example usage
image_path = '/Users/levi/Downloads/Levi.jpg'  # Replace with your image path
image = load_image(image_path)

large_img = image
small_to_large_image_size_ratio = 0.1
small_img = cv2.resize(large_img, # original image
                       (0,0), # set fx and fy, not the final size
                       fx=small_to_large_image_size_ratio, 
                       fy=small_to_large_image_size_ratio, 
                       interpolation=cv2.INTER_NEAREST)

# plt.imshow(small_img, cmap='gray')

dual_graph = construct_dual_graph(small_img)

# Compute the MST and visualize it
mst = compute_minimum_spanning_tree(dual_graph)
visualize_graph(mst, image.shape)

# Perform the Manhattan traversal and plot the result
path = manhattan_traversal(mst)
plot_path(small_img, path)


In [44]:
import torch
import numpy as np

def generate_spiral_indices(H, W):
    # Create an empty list to store the indices in spiral order
    indices = []
    
    left, right, top, bottom = 0, W - 1, 0, H - 1

    while left <= right and top <= bottom:
        # Traverse from left to right
        for i in range(left, right + 1):
            indices.append(top * W + i)
        top += 1

        # Traverse downwards
        for i in range(top, bottom + 1):
            indices.append(i * W + right)
        right -= 1

        if top <= bottom:
            # Traverse from right to left
            for i in range(right, left - 1, -1):
                indices.append(bottom * W + i)
            bottom -= 1

        if left <= right:
            # Traverse upwards
            for i in range(bottom, top - 1, -1):
                indices.append(i * W + left)
            left += 1

    return torch.tensor(indices, dtype=torch.long)

# Example usage
H, W = 64, 64
spiral_indices = generate_spiral_indices(H, W)
column_vector = np.arange(64*64)

matrix = np.zeros((64*64, 64*64), dtype=int)

matrix[column_vector, spiral_indices] = 1

# np.save('spiral_eye.npy', matrix)
# np.save('despiral_eye.npy', np.transpose(matrix))

# spiral_indices_r = spiral_indices.flip(0)
# matrix = np.zeros((64*64, 64*64), dtype=int)

# matrix[column_vector, spiral_indices_r] = 1

# np.save('despiral_r_eye.npy', np.transpose(matrix))

In [45]:
print(column_vector.shape)
print(spiral_indices.shape)

(4096,)
torch.Size([4096])


In [46]:
import torch

In [47]:
torch.manual_seed(0)

A = torch.randint(0, 3, (5, 5))
B = torch.randint(0, 3, (1, 1, 5))

print(A)
print(B)

#4096x4096 (ij) multiplied by 1x1x4096 (klj) to make 1x1x4096 (kli)

C = torch.einsum('ij,klj->kli', A, B)

C_2 = torch.einsum('ij,klj->klj', A, B)

# Check if two are equal
print(C == C_2)

print(C)
print(C_2)


tensor([[2, 0, 2, 0, 1],
        [0, 1, 1, 1, 0],
        [2, 2, 0, 0, 1],
        [2, 0, 0, 2, 0],
        [2, 2, 2, 2, 2]])
tensor([[[2, 1, 2, 0, 0]]])
tensor([[[False, False, False, False, False]]])
tensor([[[ 8,  3,  6,  4, 10]]])
tensor([[[16,  5, 10,  0,  0]]])


In [48]:
data = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15]]
x_data = torch.tensor(data)
print(x_data)
x_new = x_data.view(-1)
x_new = x_new.unsqueeze_(0).unsqueeze_(0).to(torch.float32)
print(x_new)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
          14., 15.]]])


In [49]:
# Example usage 2
H, W = 4, 4
spiral_indices_2 = generate_spiral_indices(H, W)
column_vector_2 = np.arange(4*4)

matrix_2 = np.zeros((4*4, 4*4), dtype=int)

matrix_2[column_vector_2, spiral_indices_2] = 1
matrix_2_tensor = torch.tensor(matrix_2, dtype=torch.float32)

In [107]:
print(spiral_indices_2)

tensor([ 0,  1,  2,  3,  7, 11, 15, 14, 13, 12,  8,  4,  5,  6, 10,  9])


In [108]:
C = torch.einsum('ij,klj->kli', matrix_2_tensor, x_new)

C_2 = torch.einsum('ij,klj->klj', matrix_2_tensor, x_new)

In [106]:
print(matrix_2_tensor)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0

In [109]:
print(x_data)
print(C)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
tensor([[[ 0.,  1.,  2.,  3.,  7., 11., 15., 14., 13., 12.,  8.,  4.,  5.,  6.,
          10.,  9.]]])


In [110]:
print(C_2)

tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
          14., 15.]]])


In [104]:
print(x_new.view(4, 4))

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])


## Gilbert curves

In [2]:
import torch
import numpy as np

In [3]:
def gilbert2d(width, height):
    """
    Generalized Hilbert ('gilbert') space-filling curve for arbitrary-sized
    2D rectangular grids. Generates discrete 2D coordinates to fill a rectangle
    of size (width x height).
    """

    if width >= height:
        yield from generate2d(0, 0, width, 0, 0, height)
    else:
        yield from generate2d(0, 0, 0, height, width, 0)


def sgn(x):
    return -1 if x < 0 else (1 if x > 0 else 0)


def generate2d(x, y, ax, ay, bx, by):

    w = abs(ax + ay)
    h = abs(bx + by)

    (dax, day) = (sgn(ax), sgn(ay)) # unit major direction
    (dbx, dby) = (sgn(bx), sgn(by)) # unit orthogonal direction

    if h == 1:
        # trivial row fill
        for i in range(0, w):
            yield(x, y)
            (x, y) = (x + dax, y + day)
        return

    if w == 1:
        # trivial column fill
        for i in range(0, h):
            yield(x, y)
            (x, y) = (x + dbx, y + dby)
        return

    (ax2, ay2) = (ax//2, ay//2)
    (bx2, by2) = (bx//2, by//2)

    w2 = abs(ax2 + ay2)
    h2 = abs(bx2 + by2)

    if 2*w > 3*h:
        if (w2 % 2) and (w > 2):
            # prefer even steps
            (ax2, ay2) = (ax2 + dax, ay2 + day)

        # long case: split in two parts only
        yield from generate2d(x, y, ax2, ay2, bx, by)
        yield from generate2d(x+ax2, y+ay2, ax-ax2, ay-ay2, bx, by)

    else:
        if (h2 % 2) and (h > 2):
            # prefer even steps
            (bx2, by2) = (bx2 + dbx, by2 + dby)

        # standard case: one step up, one long horizontal, one step down
        yield from generate2d(x, y, bx2, by2, ax2, ay2)
        yield from generate2d(x+bx2, y+by2, ax, ay, bx-bx2, by-by2)
        yield from generate2d(x+(ax-dax)+(bx2-dbx), y+(ay-day)+(by2-dby),
                              -bx2, -by2, -(ax-ax2), -(ay-ay2))

In [71]:
generator = gilbert2d(4, 4)

In [68]:
for i in generator:
    print(i)

(0, 0)
(1, 0)
(1, 1)
(0, 1)
(0, 2)
(0, 3)
(1, 3)
(1, 2)
(2, 2)
(2, 3)
(3, 3)
(3, 2)
(3, 1)
(2, 1)
(2, 0)
(3, 0)


In [58]:
print(x_new.view(4, 4))

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])


In [60]:
data_rectangle = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]
x_data_rectangle = torch.tensor(data_rectangle)
print(x_data_rectangle)
print(x_data_rectangle.shape)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])
torch.Size([4, 3])


In [ ]:
def generate_gilbert_indices(H, W, generator):
    indices = []
    # Origin is bottom left corner of tensor
    origin = (W*H - H)
    currentPos = origin
    # Iterate through coordinates in the generator
    for i in generator:
        print(i)
        # If the current position is the origin, add it to the indices
        if i == (0, 0):
            indices.append(origin)
            currentXY = i
        # Moving right in x direction
        elif currentXY[0]+1 == i[0]:
            currentPos += 1
            indices.append(currentPos)
            currentXY = i
        # Moving left in x direction
        elif currentXY[0]-1 == i[0]:
            currentPos -= 1
            indices.append(currentPos)
            currentXY = i
        # Moving up in y direction
        elif currentXY[1]+1 == i[1]:
            currentPos -= W
            indices.append(currentPos)
            currentXY = i
        # Moving down in y direction
        elif currentXY[1]-1 == i[1]:
            currentPos += W
            indices.append(currentPos)
            currentXY = i
    return torch.tensor(indices, dtype=torch.long)
        

In [73]:
myList = generate_gilbert_indices(4, 4, generator)

(0, 0)
(1, 0)
(1, 1)
(0, 1)
(0, 2)
(0, 3)
(1, 3)
(1, 2)
(2, 2)
(2, 3)
(3, 3)
(3, 2)
(3, 1)
(2, 1)
(2, 0)
(3, 0)


In [74]:
print(x_new.view(4, 4))
print(myList)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])
tensor([12, 13,  9,  8,  4,  0,  1,  5,  6,  2,  3,  7, 11, 10, 14, 15])


## 3D Gilbert

In [118]:
data = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15]]
data2 = [[16, 17, 18, 19], [20, 21, 22, 23], [24, 25, 26, 27], [28, 29, 30, 31]]


x_data_3d = torch.tensor([data, data2])
print(x_data_3d.shape)
x_new_3d = x_data_3d.view(-1)
x_new_3d = x_new_3d.unsqueeze_(0).unsqueeze_(0).to(torch.float32)
print(x_new_3d.shape)

torch.Size([2, 4, 4])
torch.Size([1, 1, 32])


In [119]:
print(x_data_3d)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]]])


In [120]:
print(x_new_3d)

tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
          14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
          28., 29., 30., 31.]]])


In [121]:
def gilbert3d(width, height, depth):
    """
    Generalized Hilbert ('Gilbert') space-filling curve for arbitrary-sized
    3D rectangular grids. Generates discrete 3D coordinates to fill a cuboid
    of size (width x height x depth). Even sizes are recommended in 3D.
    """

    if width >= height and width >= depth:
       yield from generate3d(0, 0, 0,
                             width, 0, 0,
                             0, height, 0,
                             0, 0, depth)

    elif height >= width and height >= depth:
       yield from generate3d(0, 0, 0,
                             0, height, 0,
                             width, 0, 0,
                             0, 0, depth)

    else: # depth >= width and depth >= height
       yield from generate3d(0, 0, 0,
                             0, 0, depth,
                             width, 0, 0,
                             0, height, 0)


def sgn(x):
    return -1 if x < 0 else (1 if x > 0 else 0)


def generate3d(x, y, z,
               ax, ay, az,
               bx, by, bz,
               cx, cy, cz):

    w = abs(ax + ay + az)
    h = abs(bx + by + bz)
    d = abs(cx + cy + cz)

    (dax, day, daz) = (sgn(ax), sgn(ay), sgn(az)) # unit major direction ("right")
    (dbx, dby, dbz) = (sgn(bx), sgn(by), sgn(bz)) # unit ortho direction ("forward")
    (dcx, dcy, dcz) = (sgn(cx), sgn(cy), sgn(cz)) # unit ortho direction ("up")

    # trivial row/column fills
    if h == 1 and d == 1:
        for i in range(0, w):
            yield(x, y, z)
            (x, y, z) = (x + dax, y + day, z + daz)
        return

    if w == 1 and d == 1:
        for i in range(0, h):
            yield(x, y, z)
            (x, y, z) = (x + dbx, y + dby, z + dbz)
        return

    if w == 1 and h == 1:
        for i in range(0, d):
            yield(x, y, z)
            (x, y, z) = (x + dcx, y + dcy, z + dcz)
        return

    (ax2, ay2, az2) = (ax//2, ay//2, az//2)
    (bx2, by2, bz2) = (bx//2, by//2, bz//2)
    (cx2, cy2, cz2) = (cx//2, cy//2, cz//2)

    w2 = abs(ax2 + ay2 + az2)
    h2 = abs(bx2 + by2 + bz2)
    d2 = abs(cx2 + cy2 + cz2)

    # prefer even steps
    if (w2 % 2) and (w > 2):
       (ax2, ay2, az2) = (ax2 + dax, ay2 + day, az2 + daz)

    if (h2 % 2) and (h > 2):
       (bx2, by2, bz2) = (bx2 + dbx, by2 + dby, bz2 + dbz)

    if (d2 % 2) and (d > 2):
       (cx2, cy2, cz2) = (cx2 + dcx, cy2 + dcy, cz2 + dcz)

    # wide case, split in w only
    if (2*w > 3*h) and (2*w > 3*d):
       yield from generate3d(x, y, z,
                             ax2, ay2, az2,
                             bx, by, bz,
                             cx, cy, cz)

       yield from generate3d(x+ax2, y+ay2, z+az2,
                             ax-ax2, ay-ay2, az-az2,
                             bx, by, bz,
                             cx, cy, cz)

    # do not split in d
    elif 3*h > 4*d:
       yield from generate3d(x, y, z,
                             bx2, by2, bz2,
                             cx, cy, cz,
                             ax2, ay2, az2)

       yield from generate3d(x+bx2, y+by2, z+bz2,
                             ax, ay, az,
                             bx-bx2, by-by2, bz-bz2,
                             cx, cy, cz)

       yield from generate3d(x+(ax-dax)+(bx2-dbx),
                             y+(ay-day)+(by2-dby),
                             z+(az-daz)+(bz2-dbz),
                             -bx2, -by2, -bz2,
                             cx, cy, cz,
                             -(ax-ax2), -(ay-ay2), -(az-az2))

    # do not split in h
    elif 3*d > 4*h:
       yield from generate3d(x, y, z,
                             cx2, cy2, cz2,
                             ax2, ay2, az2,
                             bx, by, bz)

       yield from generate3d(x+cx2, y+cy2, z+cz2,
                             ax, ay, az,
                             bx, by, bz,
                             cx-cx2, cy-cy2, cz-cz2)

       yield from generate3d(x+(ax-dax)+(cx2-dcx),
                             y+(ay-day)+(cy2-dcy),
                             z+(az-daz)+(cz2-dcz),
                             -cx2, -cy2, -cz2,
                             -(ax-ax2), -(ay-ay2), -(az-az2),
                             bx, by, bz)

    # regular case, split in all w/h/d
    else:
       yield from generate3d(x, y, z,
                             bx2, by2, bz2,
                             cx2, cy2, cz2,
                             ax2, ay2, az2)

       yield from generate3d(x+bx2, y+by2, z+bz2,
                             cx, cy, cz,
                             ax2, ay2, az2,
                             bx-bx2, by-by2, bz-bz2)

       yield from generate3d(x+(bx2-dbx)+(cx-dcx),
                             y+(by2-dby)+(cy-dcy),
                             z+(bz2-dbz)+(cz-dcz),
                             ax, ay, az,
                             -bx2, -by2, -bz2,
                             -(cx-cx2), -(cy-cy2), -(cz-cz2))

       yield from generate3d(x+(ax-dax)+bx2+(cx-dcx),
                             y+(ay-day)+by2+(cy-dcy),
                             z+(az-daz)+bz2+(cz-dcz),
                             -cx, -cy, -cz,
                             -(ax-ax2), -(ay-ay2), -(az-az2),
                             bx-bx2, by-by2, bz-bz2)

       yield from generate3d(x+(ax-dax)+(bx2-dbx),
                             y+(ay-day)+(by2-dby),
                             z+(az-daz)+(bz2-dbz),
                             -bx2, -by2, -bz2,
                             cx2, cy2, cz2,
                             -(ax-ax2), -(ay-ay2), -(az-az2))

In [122]:
generator3d = gilbert3d(4, 4, 2)

In [86]:
# for i in generator3d:
#     print(i)

In [ ]:
def generate_gilbert_indices_3D(H, W, D, generator):
    indices = []
    # Origin is bottom left corner of tensor
    origin = (W*H - H)
    currentPos = origin
    # Iterate through coordinates in the generator
    for i in generator:
        # If the current position is the origin, add it to the indices
        if i == (0, 0, 0):
            indices.append(origin)
            currentXYZ = i
        # Moving right in x direction
        elif currentXYZ[0]+1 == i[0]:
            currentPos += 1
            indices.append(currentPos)
            currentXYZ = i
        # Moving left in x direction
        elif currentXYZ[0]-1 == i[0]:
            currentPos -= 1
            indices.append(currentPos)
            currentXYZ = i
        # Moving up in y direction
        elif currentXYZ[1]+1 == i[1]:
            currentPos -= W
            indices.append(currentPos)
            currentXYZ = i
        # Moving down in y direction
        elif currentXYZ[1]-1 == i[1]:
            currentPos += W
            indices.append(currentPos)
            currentXYZ = i
        # Moving forward in z direction
        elif currentXYZ[2]+1 == i[2]:
            currentPos += H*W
            indices.append(currentPos)
            currentXYZ = i
        # Moving backward in z direction
        elif currentXYZ[2]-1 == i[2]:
            currentPos -= H*W
            indices.append(currentPos)
            currentXYZ = i
    return torch.tensor(indices, dtype=torch.long)
        

In [135]:
myList3D = generate_gilbert_indices_3D(4, 4, 2, generator3d)

In [126]:
print(x_data_3d)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]]])


In [129]:
print(myList3D)
print(x_new_3d)

tensor([12, 28, 29, 13,  9, 25, 24,  8,  4,  0, 16, 20, 21, 17,  1,  5,  6,  2,
        18, 22, 23, 19,  3,  7, 11, 27, 26, 10, 14, 30, 31, 15])
tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
          14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
          28., 29., 30., 31.]]])


In [131]:
H, W, D = 4, 4, 2
column_vector3D = np.arange(4*4*2)

matrix3D = np.zeros((4*4*2, 4*4*2), dtype=int)
print(column_vector3D.shape)
print(myList3D.shape)
print(matrix3D.shape)

matrix3D[column_vector3D, myList3D] = 1
matrix_3D_tensor = torch.tensor(matrix3D, dtype=torch.float32)

(32,)
torch.Size([32])
(32, 32)


In [ ]:
C_3D = torch.einsum('ij,klj->kli', matrix_3D_tensor, x_new_3d) # Good

C_2_3D = torch.einsum('ij,klj->klj', matrix_3D_tensor, x_new_3d) # Incorrect

In [ ]:
print(myList3D)
print(C_3D)
print(C_2_3D)

tensor([12, 28, 29, 13,  9, 25, 24,  8,  4,  0, 16, 20, 21, 17,  1,  5,  6,  2,
        18, 22, 23, 19,  3,  7, 11, 27, 26, 10, 14, 30, 31, 15])
tensor([[[12., 28., 29., 13.,  9., 25., 24.,  8.,  4.,  0., 16., 20., 21., 17.,
           1.,  5.,  6.,  2., 18., 22., 23., 19.,  3.,  7., 11., 27., 26., 10.,
          14., 30., 31., 15.]]])
tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
          14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
          28., 29., 30., 31.]]])


## Slice-Spiral

In [159]:
def generate_slicewise_spiral_indices(H, W, D):
    # Create an empty list to store the indices in spiral order
    indices = []
    
    left, right, top, bottom = 0, W - 1, 0, H - 1

    while left <= right and top <= bottom:
        # Traverse from left to right
        for i in range(left, right + 1):
            indices.append(top * W + i)
        top += 1

        # Traverse downwards
        for i in range(top, bottom + 1):
            indices.append(i * W + right)
        right -= 1

        if top <= bottom:
            # Traverse from right to left
            for i in range(right, left - 1, -1):
                indices.append(bottom * W + i)
            bottom -= 1

        if left <= right:
            # Traverse upwards
            for i in range(bottom, top - 1, -1):
                indices.append(i * W + left)
            left += 1
    
    indices_slice1 = indices[:]

    for i in range(D-1):
        if i%2 == 0:
            indicesNew = [x+(H*W*(i+1)) for x in list(reversed(indices_slice1))]
        else:
            indicesNew = [x+(H*W*(i+1)) for x in indices_slice1]

        indices += indicesNew
    
    return torch.tensor(indices, dtype=torch.long)

In [170]:
data = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15]]
data2 = [[16, 17, 18, 19], [20, 21, 22, 23], [24, 25, 26, 27], [28, 29, 30, 31]]
data3 = [[32, 33, 34, 35], [36, 37, 38, 39], [40, 41, 42, 43], [44, 45, 46, 47]]

x_data_3d = torch.tensor([data, data2, data3])
print(x_data_3d.shape)
x_new_3d = x_data_3d.view(-1)
x_new_3d = x_new_3d.unsqueeze_(0).unsqueeze_(0).to(torch.float32)
print(x_new_3d.shape)

torch.Size([3, 4, 4])
torch.Size([1, 1, 48])


In [171]:
spiral_indices_slice = generate_slicewise_spiral_indices(4, 4, 3)
column_vector_slice = np.arange(4*4*3)

matrix_slice = np.zeros((4*4*3, 4*4*3), dtype=int)

matrix_slice[column_vector_slice, spiral_indices_slice] = 1
matrix_slice_tensor = torch.tensor(matrix_slice, dtype=torch.float32)

In [172]:
print((spiral_indices_slice))
print(x_data_3d[:, :, :])

tensor([ 0,  1,  2,  3,  7, 11, 15, 14, 13, 12,  8,  4,  5,  6, 10,  9, 25, 26,
        22, 21, 20, 24, 28, 29, 30, 31, 27, 23, 19, 18, 17, 16, 32, 33, 34, 35,
        39, 43, 47, 46, 45, 44, 40, 36, 37, 38, 42, 41])
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39],
         [40, 41, 42, 43],
         [44, 45, 46, 47]]])


In [173]:
# list1 = [1, 2, 3, 4]
# list2 = [x+4 for x in list(reversed(list1))]
# print(list1)
# print(list2)

In [174]:
# depth = 4
# for i in range(depth):
#     print(i, i%2==0)

## Slice-Hilbert

In [175]:
def gilbert2d(width, height):
    """
    Generalized Hilbert ('gilbert') space-filling curve for arbitrary-sized
    2D rectangular grids. Generates discrete 2D coordinates to fill a rectangle
    of size (width x height).
    """

    if width >= height:
        yield from generate2d(0, 0, width, 0, 0, height)
    else:
        yield from generate2d(0, 0, 0, height, width, 0)


def sgn(x):
    return -1 if x < 0 else (1 if x > 0 else 0)


def generate2d(x, y, ax, ay, bx, by):

    w = abs(ax + ay)
    h = abs(bx + by)

    (dax, day) = (sgn(ax), sgn(ay)) # unit major direction
    (dbx, dby) = (sgn(bx), sgn(by)) # unit orthogonal direction

    if h == 1:
        # trivial row fill
        for i in range(0, w):
            yield(x, y)
            (x, y) = (x + dax, y + day)
        return

    if w == 1:
        # trivial column fill
        for i in range(0, h):
            yield(x, y)
            (x, y) = (x + dbx, y + dby)
        return

    (ax2, ay2) = (ax//2, ay//2)
    (bx2, by2) = (bx//2, by//2)

    w2 = abs(ax2 + ay2)
    h2 = abs(bx2 + by2)

    if 2*w > 3*h:
        if (w2 % 2) and (w > 2):
            # prefer even steps
            (ax2, ay2) = (ax2 + dax, ay2 + day)

        # long case: split in two parts only
        yield from generate2d(x, y, ax2, ay2, bx, by)
        yield from generate2d(x+ax2, y+ay2, ax-ax2, ay-ay2, bx, by)

    else:
        if (h2 % 2) and (h > 2):
            # prefer even steps
            (bx2, by2) = (bx2 + dbx, by2 + dby)

        # standard case: one step up, one long horizontal, one step down
        yield from generate2d(x, y, bx2, by2, ax2, ay2)
        yield from generate2d(x+bx2, y+by2, ax, ay, bx-bx2, by-by2)
        yield from generate2d(x+(ax-dax)+(bx2-dbx), y+(ay-day)+(by2-dby),
                              -bx2, -by2, -(ax-ax2), -(ay-ay2))

In [176]:
data = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15]]
data2 = [[16, 17, 18, 19], [20, 21, 22, 23], [24, 25, 26, 27], [28, 29, 30, 31]]
data3 = [[32, 33, 34, 35], [36, 37, 38, 39], [40, 41, 42, 43], [44, 45, 46, 47]]

x_data_3d = torch.tensor([data, data2, data3])
print(x_data_3d.shape)
x_new_3d = x_data_3d.view(-1)
x_new_3d = x_new_3d.unsqueeze_(0).unsqueeze_(0).to(torch.float32)
print(x_new_3d.shape)

torch.Size([3, 4, 4])
torch.Size([1, 1, 48])


In [183]:
generator = gilbert2d(4, 4)

In [ ]:
def generate_slicewise_hilbert_indices(H, W, D, generator):
    indices = []
    # Origin is bottom left corner of tensor
    origin = (W*H - H)
    currentPos = origin
    # Iterate through coordinates in the generator
    for i in generator:
        # If the current position is the origin, add it to the indices
        if i == (0, 0):
            indices.append(origin)
            currentXY = i
        # Moving right in x direction
        elif currentXY[0]+1 == i[0]:
            currentPos += 1
            indices.append(currentPos)
            currentXY = i
        # Moving left in x direction
        elif currentXY[0]-1 == i[0]:
            currentPos -= 1
            indices.append(currentPos)
            currentXY = i
        # Moving up in y direction
        elif currentXY[1]+1 == i[1]:
            currentPos -= W
            indices.append(currentPos)
            currentXY = i
        # Moving down in y direction
        elif currentXY[1]-1 == i[1]:
            currentPos += W
            indices.append(currentPos)
            currentXY = i

    indices_slice1 = indices[:]

    for i in range(D-1):
        if i%2 == 0:
            indicesNew = [x+(H*W*(i+1)) for x in list(reversed(indices_slice1))]
        else:
            indicesNew = [x+(H*W*(i+1)) for x in indices_slice1]
        indices += indicesNew

    return torch.tensor(indices, dtype=torch.long)
        

In [ ]:
myList = generate_slicewise_hilbert_indices(4, 4, 3, generator)

In [187]:
print(myList)
print(x_data_3d[:, :, :])

tensor([12, 13,  9,  8,  4,  0,  1,  5,  6,  2,  3,  7, 11, 10, 14, 15, 31, 30,
        26, 27, 23, 19, 18, 22, 21, 17, 16, 20, 24, 25, 29, 28, 44, 45, 41, 40,
        36, 32, 33, 37, 38, 34, 35, 39, 43, 42, 46, 47])
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39],
         [40, 41, 42, 43],
         [44, 45, 46, 47]]])
